In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
import scipy.stats as sp
import scipy.optimize as sco
import plotly
from plotly import graph_objs as go
plotly.offline.init_notebook_mode(connected = True)

In [4]:
def BSM(S0, K, tau, r, sigma, opt_type='c', q=0):
    d1 = (np.log(S0/K)+(r-q+.5*sigma**2)*tau)/(sigma*np.sqrt(tau))
    d2 = d1 - sigma*np.sqrt(tau)
    N = lambda x: sp.norm.cdf(x)
    if opt_type == 'c':
        return S0*np.exp(-q*tau)*N(d1) - np.exp(-r*tau)*K*N(d2)
    else:
        return K*np.exp(-r*tau)*N(-d2) - S0*np.exp(-q*tau)*N(-d1)

def bisection(f, a, b, tol=1e-6):
    c = (a+b)/2
    while(np.abs(b-a)>tol):
        if f(c) == 0:
            return c
        elif f(a)*f(c)<0:
            b=c
        else:
            a=c
        c = (a+b)/2
    return c

def impliedVol(S0, K, tau, r, price, opt_type='c'):
    equation = lambda sigma: BSM(S0,K,tau,r,sigma,opt_type) - price
    return bisection(equation,-1,5)

Data was downloaded 27-Apr-2018 at 3:45pm

In [5]:
files = ['20180427', '20180504', '20180511', '20180518', '20180525', '20180601', '20180608', '20180615',
        '20180720', '20180817', '20180921', '20181019', '20181116', '20190118', '20190215', '20190621',
        '20200117']
pd.to_datetime(files)

DatetimeIndex(['2018-04-27', '2018-05-04', '2018-05-11', '2018-05-18',
               '2018-05-25', '2018-06-01', '2018-06-08', '2018-06-15',
               '2018-07-20', '2018-08-17', '2018-09-21', '2018-10-19',
               '2018-11-16', '2019-01-18', '2019-02-15', '2019-06-21',
               '2020-01-17'],
              dtype='datetime64[ns]', freq=None)

In [6]:
dfs = []
for f in files:
    temp = pd.read_csv('{}.csv'.format(f))
    dfs.append(temp)
dfs

[     Unnamed: 0         T       K   P_bid   P_ask   C_bid   C_ask        S  \
 0             0  20180427   910.0    0.00    0.02  607.10  611.80  1517.96   
 1             1  20180427   920.0    0.00    0.03  596.50  601.50  1517.96   
 2             2  20180427   930.0    0.00    3.80  586.70  591.70  1517.96   
 3             3  20180427   940.0    0.00    0.05  576.50  581.50  1517.96   
 4             4  20180427   950.0    0.00    0.05  566.50  571.50  1517.96   
 5             5  20180427   960.0    0.00    1.67  556.40  561.40  1517.96   
 6             6  20180427   970.0    0.00    3.35  546.50  551.50  1517.96   
 7             7  20180427   980.0    0.00    4.25  536.50  541.50  1517.96   
 8             8  20180427   990.0    0.00    0.68  526.55  531.55  1517.96   
 9             9  20180427  1000.0    0.00    0.04  516.55  521.55  1517.96   
 10           10  20180427  1010.0    0.00    0.68  506.55  511.55  1517.96   
 11           11  20180427  1020.0    0.00    0.08  

In [7]:
r = 0.0184 # 3m Treasury 4/30/18
# for df in dfs:
#     if df['T'][0] == 20180427:
#         df['TTM']= 1/365
#     else:
#         df['TTM']=(pd.to_datetime(df['T'].apply(lambda x: str(x)))-datetime(2018,4,27)).apply(lambda y: y.days/365)
#     df['P_mid'] = (df.P_bid + df.P_ask)/2
#     df['C_mid'] = (df.C_bid + df.C_ask)/2
#     df['P_bid_IV'] = df.apply(lambda x: impliedVol(x.S, x.K, x.TTM, r, x.P_bid, opt_type='p'),axis=1)
#     df['P_IV'] = df.apply(lambda x: impliedVol(x.S, x.K, x.TTM, r, x.P_mid, opt_type='p'),axis=1)
#     df['P_ask_IV'] = df.apply(lambda x: impliedVol(x.S, x.K, x.TTM, r, x.P_ask, opt_type='p'),axis=1)
#     df['C_bid_IV'] = df.apply(lambda x: impliedVol(x.S, x.K, x.TTM, r, x.C_bid, opt_type='c'),axis=1)
#     df['C_IV'] = df.apply(lambda x: impliedVol(x.S, x.K, x.TTM, r, x.C_mid, opt_type='c'),axis=1)
#     df['C_ask_IV'] = df.apply(lambda x: impliedVol(x.S, x.K, x.TTM, r, x.C_ask, opt_type='c'),axis=1)
#     df = df[['T','TTM','K','P_bid','P_mid','P_ask','P_IV','C_bid','C_mid','C_ask','C_IV','S','dS']]
dfs[0]

,Unnamed: 0,T,K,P_bid,P_ask,C_bid,C_ask,S,dS,TTM,P_mid,C_mid,P_IV,C_IV,P_bid_IV,P_ask_IV,C_bid_IV,C_ask_IV
0,0,20180427,910.0,0.00,0.02,607.10,611.80,1517.96,0.001,0.00274,0.010,609.450,2.792831e+00,4.495669,1.250000e-01,2.931172,1.192093e-07,5.000000
1,1,20180427,920.0,0.00,0.03,596.50,601.50,1517.96,0.001,0.00274,0.015,599.000,2.813418e+00,4.191422,1.250000e-01,2.960090,1.192093e-07,5.000000
2,2,20180427,930.0,0.00,3.80,586.70,591.70,1517.96,0.001,0.00274,1.900,589.200,4.499743e+00,4.211310,1.250000e-01,5.000000,1.192093e-07,5.000000
3,3,20180427,940.0,0.00,0.05,576.50,581.50,1517.96,0.001,0.00274,0.025,579.000,2.801088e+00,4.028153,1.250000e-01,2.957603,1.192093e-07,4.863447
4,4,20180427,950.0,0.00,0.05,566.50,571.50,1517.96,0.001,0.00274,0.025,569.000,2.742732e+00,3.947705,1.250000e-01,2.896325,1.192093e-07,4.769614
5,5,20180427,960.0,0.00,1.67,556.40,561.40,1517.96,0.001,0.00274,0.835,558.900,3.782355e+00,3.814493,1.250000e-01,4.159520,1.192093e-07,4.653400
6,6,20180427,970.0,0.00,3.35,546.50,551.50,1517.96,0.001,0.00274,1.675,549.000,4.077714e+00,3.789080,1.250000e-01,4.552130,1.192093e-07,4.584540
7,7,20180427,980.0,0.00,4.25,536.50,541.50,1517.96,0.001,0.00274,2.125,539.000,4.141626e+00,3.710865,1.250000e-01,4.652976,1.192093e-07,4.493254
8,8,20180427,990.0,0.00,0.68,526.55,531.55,1517.96,0.001,0.00274,0.340,529.050,3.191054e+00,3.657488,1.250000e-01,3.461338,1.192093e-07,4.413711
9,9,20180427,1000.0,0.00,0.04,516.55,521.55,1517.96,0.001,0.00274,0.020,519.050,2.418402e+00,3.580253,1.250000e-01,2.551485,1.192093e-07,4.323835


In [33]:
# for i in range(len(dfs)):
#     dfs[i].to_csv('{}.csv'.format(files[i]))

In [6]:
for df in dfs:
    trace0 = go.Scatter(
        x = df.K, y = 50*(df.P_bid_IV + df.P_ask_IV), name = '{} Put Mid IV'.format(df['T'][0])
    )
    trace1 = go.Scatter(
        x = df.K, y = 100*df.P_IV, name = '{} Put Mid'.format(df['T'][0])
    )
    trace2 = go.Scatter(
        x = df.K, y = 50*(df.C_bid_IV + df.C_ask_IV), name = '{} Call Mid IV'.format(df['T'][0])
    )
    trace3 = go.Scatter(
        x = df.K, y = 100*df.C_IV, name = '{} Call Mid'.format(df['T'][0])
    )
    trace4 = go.Scatter(
        x = df.S, 
        y = np.linspace(100*min(np.append(df.P_IV,df.C_IV))-5,100*max(np.append(df.P_IV,df.C_IV))+5,len(df.S)), 
        name = 'Underlying'
    )
    data = [trace0,trace1,trace2,trace3,trace4]
    layout = go.Layout(
    title = 'AMZN Call and Put Implied Volatility Expiration: {}'.format(df['T'][0]), 
        yaxis = dict(title = 'Implied Volatility'), xaxis = dict(title = 'Strike'))
    fig = go.Figure(data=data, layout=layout)
    plotly.offline.iplot(fig)

In [8]:
mid1s = [] # mid price
mid2s = [] # mid IV
for df in dfs:
    temp = pd.DataFrame({'K':df['K'], 'TTM':df['TTM'], 'C_IV': df['C_IV'], 'P_IV': df['P_IV']})
    temp['C_BS_Price'] = df.apply(lambda x: BSM(x.S, x.K, x.TTM, r, x.C_IV), axis=1)
    temp['P_BS_Price'] = df.apply(lambda x: BSM(x.S, x.K, x.TTM, r, x.P_IV, 'p'), axis=1)
    mid1s.append(temp)
    temp = pd.DataFrame({'K':df['K'], 'TTM':df['TTM'], 'C_IV': (df['C_bid_IV'] + df['C_ask_IV'])/2, 'P_IV': (df['P_bid_IV'] + df['P_ask_IV'])/2})
    temp['C_BS_Price'] = df.apply(lambda x: BSM(x.S, x.K, x.TTM, r, (x['C_bid_IV'] + x['C_ask_IV'])/2), axis=1)
    temp['P_BS_Price'] = df.apply(lambda x: BSM(x.S, x.K, x.TTM, r, (x['P_bid_IV'] + x['P_ask_IV'])/2, 'p'),axis=1)
    mid2s.append(temp)
    
mid1s

[          K      TTM      C_IV          P_IV  C_BS_Price  P_BS_Price
 0     910.0  0.00274  4.495669  2.792831e+00  609.449999    0.010000
 1     920.0  0.00274  4.191422  2.813418e+00  599.000000    0.015000
 2     930.0  0.00274  4.211310  4.499743e+00  589.200000    1.900001
 3     940.0  0.00274  4.028153  2.801088e+00  579.000000    0.025000
 4     950.0  0.00274  3.947705  2.742732e+00  569.000000    0.025000
 5     960.0  0.00274  3.814493  3.782355e+00  558.900001    0.835000
 6     970.0  0.00274  3.789080  4.077714e+00  549.000000    1.675000
 7     980.0  0.00274  3.710865  4.141626e+00  539.000000    2.125000
 8     990.0  0.00274  3.657488  3.191054e+00  529.050000    0.340000
 9    1000.0  0.00274  3.580253  2.418402e+00  519.050000    0.020000
 10   1010.0  0.00274  3.503692  3.054464e+00  509.050000    0.340000
 11   1020.0  0.00274  3.512627  2.438032e+00  499.250000    0.040000
 12   1030.0  0.00274  3.435856  2.905584e+00  489.249999    0.325000
 13   1040.0  0.0027

In [7]:
def opt_w(rho,Q,wp,lam,B_r,B_t):
    n = len(rho)
    bnds = sco.Bounds(0,1,keep_feasible=True)
    w0 = n*[1/n]
    cons = ({'type':'eq','fun':lambda x: np.sum(x)-1},
            {'type':'eq','fun':lambda x: np.dot(B_r,x)-B_t})
    def obj(x):
        return -x.T.dot(rho)+lam*(x-wp).T.dot(Q.dot(x-wp))
    result = sco.minimize(obj,w0,method='SLSQP',constraints=cons,bounds=bnds)
    return result.x

def solve_NS_params(df):
    obj = lambda x: calc_losses(df,x[0],x[1],x[2],x[3])
    cons = ({'type':'ineq','fun': lambda x: x[0]},
            {'type':'ineq','fun': lambda x: x[0]+x[1]},
            {'type':'ineq','fun': lambda x: x[3]})
    bnds = ((0,1),(-1,1),(-1,1),(0.001,5))
    guess = [.1,.1,.1,2]
    res = sco.minimize(obj,guess,constraints=cons,bounds=bnds,
                       tol=1e-12,options={'disp':True})
    return res['x']

In [12]:
np.asarray(mid1s[0].K)

array([ 910. ,  920. ,  930. ,  940. ,  950. ,  960. ,  970. ,  980. ,
        990. , 1000. , 1010. , 1020. , 1030. , 1040. , 1050. , 1060. ,
       1070. , 1080. , 1090. , 1100. , 1110. , 1120. , 1125. , 1130. ,
       1135. , 1140. , 1145. , 1150. , 1155. , 1160. , 1165. , 1170. ,
       1175. , 1180. , 1185. , 1190. , 1195. , 1200. , 1205. , 1210. ,
       1215. , 1220. , 1225. , 1227.5, 1230. , 1232.5, 1235. , 1237.5,
       1240. , 1242.5, 1245. , 1247.5, 1250. , 1252.5, 1255. , 1257.5,
       1260. , 1262.5, 1265. , 1267.5, 1270. , 1272.5, 1275. , 1277.5,
       1280. , 1282.5, 1285. , 1287.5, 1290. , 1292.5, 1295. , 1297.5,
       1300. , 1302.5, 1305. , 1307.5, 1310. , 1312.5, 1315. , 1317.5,
       1320. , 1322.5, 1325. , 1327.5, 1330. , 1332.5, 1335. , 1337.5,
       1340. , 1342.5, 1345. , 1347.5, 1350. , 1352.5, 1355. , 1357.5,
       1360. , 1362.5, 1365. , 1367.5, 1370. , 1372.5, 1375. , 1377.5,
       1380. , 1382.5, 1385. , 1387.5, 1390. , 1392.5, 1395. , 1397.5,
      

In [9]:
tol = 10E-9
def SVI(df,a,b,p,m,sig):
    k = np.log(np.asarray(df.K)/(1517.96*np.exp(r*np.asarray(df.TTM))))
    return a + b*(p*(np.asarray(k)-m) + np.sqrt((np.asarray(k)-m)**2 + sig**2))
def SVI_fit(df,IV_col):
    obj = lambda x: np.sum((SVI(df, x[0], x[1], x[2], x[3], x[4]) - np.asarray(df[IV_col]))**2)
    cons = ({'type':'ineq','fun': lambda x: x[0] + x[1]*x[4]*np.sqrt(1 - x[2]**2)},
            {'type':'ineq','fun': lambda x: x[1]},
            {'type':'ineq','fun': lambda x: x[4] - tol})
    bnds = ((None,None),(0,None),(-1+tol,1-tol),(None,None),(tol,None))
    guess = [.01,.01,.01,.01,.01]
    result = sco.minimize(obj,guess,constraints=cons,bounds=bnds,tol=tol)
    return result

SVI_fit(mid1s[0],'C_IV')

     fun: 2.8959176165665372
     jac: array([ 1.58607960e-04,  4.78029251e-05,  5.02586365e-04, -7.84397125e-04,
        2.24313140e-03])
 message: 'Optimization terminated successfully.'
    nfev: 632
     nit: 86
    njev: 86
  status: 0
 success: True
       x: array([-4.12333724, 11.5849291 , -0.07501159,  0.02251262,  0.45335302])

In [10]:
for i in range(len(mid1s)):
    temp = SVI_fit(mid1s[i],'C_IV')['x']
    trace0 = go.Scatter(
        x = np.log(np.asarray(mid1s[i].K)/(1517.96*np.exp(r*np.asarray(mid1s[i].TTM)))), y = mid1s[i].C_IV, 
        name = 'Call IV'
    )
    trace1 = go.Scatter(
        x = np.log(np.asarray(mid1s[i].K)/(1517.96*np.exp(r*np.asarray(mid1s[i].TTM)))), 
        y = SVI(mid1s[i],temp[0],temp[1],temp[2],temp[3],temp[4]), name = 'Raw SVI Fit'
    )
    data = [trace0,trace1]
    layout = go.Layout(
        title = 'AMZN Call Raw SVI Date: {}'.format(files[i]), 
            yaxis = dict(title = 'Implied Volatility (%)'), xaxis = dict(title = 'Normalized Strike'))
    fig = go.Figure(data=data, layout=layout)
    plotly.offline.iplot(fig)

     fun: 0.03143952628843545
     jac: array([-2.88377050e-04, -6.08642586e-05, -8.03918578e-04,  1.03436690e-03,
       -3.12410295e-04])
 message: 'Optimization terminated successfully.'
    nfev: 407
     nit: 55
    njev: 55
  status: 0
 success: True
       x: array([-0.15122195,  3.91817666,  0.79295307,  0.36946765,  0.21276926])